In [ ]:
%%pyspark
df = spark.read.load('abfss://staging@raritbiprod.dfs.core.windows.net/Data/Products/products.csv', format='csv'
## If header exists uncomment line below
, header=True
)
display(df.limit(10))

# Load the file data into a delta table
 delta_table_path = "/Data/Products"
 df.write.format("delta").save(delta_table_path)


In [ ]:
 from delta.tables import *
 from pyspark.sql.functions import *

 # Create a deltaTable object
 deltaTable = DeltaTable.forPath(spark, delta_table_path)

 # Update the table (reduce price of product 771 by 10%)
 deltaTable.update(
     condition = "ProductID == 771",
     set = { "ListPrice": "ListPrice * 0.9" })

 # View the updated data as a dataframe
 deltaTable.toDF().show(10)

In [ ]:
 new_df = spark.read.format("delta").load(delta_table_path)
 new_df.show(10)

Modify the code you just ran as follows, specifying the option to use the time travel feature of delta lake to view a previous version of the data.

In [ ]:
 new_df = spark.read.format("delta").option("versionAsOf", 0).load(delta_table_path)
 new_df.show(10)

To view the history

In [ ]:
 deltaTable.history(10).show(20, False, True)

In [ ]:
# Create catalog tables
# Create an external table

spark.sql("CREATE DATABASE AdventureWorks")
spark.sql("CREATE TABLE AdventureWorks.ProductsExternal USING DELTA LOCATION '{0}'".format(delta_table_path))
spark.sql("DESCRIBE EXTENDED AdventureWorks.ProductsExternal").show(truncate=False)

In [ ]:
# Read the tables you created

 %%sql

 USE AdventureWorks;

 SELECT * FROM ProductsExternal;

In [ ]:
# Create a managed table

 df.write.format("delta").saveAsTable("AdventureWorks.ProductsManaged")
 spark.sql("DESCRIBE EXTENDED AdventureWorks.ProductsManaged").show(truncate=False)

In [ ]:
# List tables
 %%sql

 USE AdventureWorks;

 SHOW TABLES;

In [ ]:
# Create a table using SQL

 %%sql

 USE AdventureWorks;

 CREATE TABLE Products
 USING DELTA
 LOCATION '/delta/products-delta';